# Fine-tuning BERT for Binary Classification



In [1]:
# Mount the Google drive for access to files
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# Basic Python modules
import os
import re
from collections import defaultdict, Counter
import random
import pickle

# For data manipulation and analysis
import pandas as pd
import numpy as np

# For machine learning tools and evaluation
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# For deep learning
# https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
import torch

In [3]:
incerto_dir = '/content/drive/MyDrive/incerto-autore'
new_poems_dir = os.path.join(incerto_dir, 'data', 'poems')
poems_split_df = pd.read_csv(os.path.join(new_poems_dir, 'poems_split.csv'))
len(poems_split_df)

682

In [4]:
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.9 MB/s eta 0:00:00


In [5]:
# using DistilBERT for testing --> can switch to BERT once set up
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

In [6]:
# Choose the GPU we want to process this script
device_name = 'cuda'      

# Choose the BERT model that we want to use (make sure to keep the cased/uncased consistent)
model = 'dbmdz/bert-base-italian-xxl-uncased'
#model = os.path.join(incerto_dir, 'contbertoldo-all', 'checkpoint')

# This is the maximum number of tokens in any document sent to BERT
max_length = 512                                                        

In [7]:
if 'contbertoldo' in model:
  finetuned_path = os.path.join(incerto_dir, 'output','finetuned-models', 'binary-class', 'bertoldo')
elif 'italian':
  finetuned_path = os.path.join(incerto_dir, 'output','finetuned-models', 'binary-class', 'bert-ita')
if not os.path.exists(finetuned_path):
  os.makedirs(finetuned_path)

### BERT setup

In [8]:
# load the encoder/tokenizer
tokenizer = BertTokenizer.from_pretrained(model)

In [9]:
# class for Torch dataset
class SCDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [10]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    learning_rate=5e-5,              # initial learning rate for Adam optimizer
    warmup_steps=50,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy='steps',
)

In [11]:
# load pre-trained model
model = BertForSequenceClassification.from_pretrained(model).to(device_name)

Some weights of the model checkpoint at dbmdz/bert-base-italian-xxl-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification

In [12]:
# Define a custom evaluation function (this could be changes to return accuracy metrics)
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

## Classification task setup

In [13]:
annotations_df = poems_split_df.loc[poems_split_df['author'] != 'Unknown']
len(annotations_df)

622

In [14]:
for author in annotations_df['author'].unique():

  print(author)
  author_finetuned_path = os.path.join(finetuned_path, author)
  if not os.path.exists(author_finetuned_path):
    os.makedirs(author_finetuned_path)

    X = annotations_df['poem'].tolist()
    y = annotations_df['author'].map(lambda x: 1 if x==author else 0).tolist()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
    print('Y train', Counter(y_train))
    print('Y test', Counter(y_test))
    print(X_test[0:3])

    # Pass training/testing sentences to tokenizer, truncate them if over max length, and add padding (PAD tokens up to 512)
    train_encodings = tokenizer(X_train,  truncation=True, padding=True)
    test_encodings = tokenizer(X_test,  truncation=True, padding=True)

    # Combine encoded text and labels into a torch dataset object.
    train_dataset = SCDataset(train_encodings, y_train)
    test_dataset = SCDataset(test_encodings, y_test)

    # Create the trainer object based on what we've set up prior to this point! This combines our model, training_args, train_dataset and test_dataset, and custom evaluation function compute_metrics.
    trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataset,         # training dataset
        eval_dataset=test_dataset,            # evaluation dataset
        compute_metrics=compute_metrics      # custom evaluation function
    )

    # Fine-tune the model on our dataset/labels. The trainer object will periodically output the state of the model.
    trainer.train()

    # built in evaluation function
    trainer.evaluate()

    #save model
    model.save_pretrained(author_finetuned_path)

    print(Counter(y_test))

    predicted_labels = trainer.predict(test_dataset)
    actual_predicted_labels = predicted_labels.predictions.argmax(-1)

    class_report = classification_report(predicted_labels.label_ids.flatten(), actual_predicted_labels.flatten(), output_dict=True)
    print(classification_report(predicted_labels.label_ids.flatten(), actual_predicted_labels.flatten()))

    # New + simple save of classification report
    class_report_df = pd.DataFrame(class_report).transpose()
    class_report_df.to_csv(os.path.join(author_finetuned_path, 'classification_report.csv'))

Franco
Y train Counter({0: 301, 1: 165})
Y test Counter({0: 97, 1: 59})
['Lassa che s un nemico a l altro chieda al suo bisogno aiuto ei gli vien dato che la virtu convien che gli odii ecceda e io creder devro ch aspro e ingrato esser mi debba il mio signor diletto perch ei sia forse d altra innamorato Oime che d altra standosi nel letto me lascia raffreddar sola e scontenta colma d affanni e piena di dispetto altra ei fa del suo amor lieta e contenta e del mio mal con lei fors ancor ride che vanagloriosa ne diventa Quanto per me si lagrima e si stride dolce concento e de le loro orecchie', 'Quando m aprira mai benigna Aurora Quel lieto di de l aureo albergo uscita Ch a te pur torni homai cara e gradita Moglie e rompa si lunga aspra dimora Cosi godano ogn hor quest occhi anchora L imagin bella c ho nel cor scolpita Com io privo di te mia luce e vita Non ho mai lieta o riposata un hora Tu fidissima sposa honesta e bella Sola mi giovi e cangi l pianto in riso Con l angelico volto e la fa

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
10,0.674600,0.664565,0.621795
20,0.660100,0.638483,0.621795
30,0.624700,0.616759,0.705128
40,0.561300,0.574096,0.685897
50,0.549900,0.367201,0.858974
60,0.399700,0.453650,0.807692
70,0.162800,0.397781,0.852564
80,0.238900,0.331927,0.891026
90,0.138200,0.274959,0.903846


Counter({0: 97, 1: 59})
              precision    recall  f1-score   support

           0       0.95      0.90      0.92        97
           1       0.84      0.92      0.88        59

    accuracy                           0.90       156
   macro avg       0.89      0.91      0.90       156
weighted avg       0.91      0.90      0.90       156

AntonGiacomoCorso
Y train Counter({0: 426, 1: 40})
Y test Counter({0: 137, 1: 19})
['Questa si mesta e dolorosa vita Che non posa o si ferma a guisa d onda Raporta hor quici hor quidi entro al mio core DOLCE l amato e dilettoso tempo Che lieto mi godea l alto mio sole Onde provo vivendo un aspra morte Prima vorrei che interompesse morte Quest anni ohime che in questa amara vita Viver noiando il Ciel la luna il sole E misurar le piagge e solcar l onda Nemica al mio soave e caro tempo Con questo travagliato tristo core Perche so ben che l rio pensier che il core Sempre tien desto condurami a morte', 'Benche fortuna rade volte amica Et la nemic

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       154
           1       0.00      0.00      0.00         2

    accuracy                           0.99       156
   macro avg       0.49      0.50      0.50       156
weighted avg       0.97      0.99      0.98       156

Petrarca
Y train Counter({0: 338, 1: 128})
Y test Counter({0: 121, 1: 35})
['Questa si mesta e dolorosa vita Che non posa o si ferma a guisa d onda Raporta hor quici hor quidi entro al mio core DOLCE l amato e dilettoso tempo Che lieto mi godea l alto mio sole Onde provo vivendo un aspra morte Prima vorrei che interompesse morte Quest anni ohime che in questa amara vita Viver noiando il Ciel la luna il sole E misurar le piagge e solcar l onda Nemica al mio soave e caro tempo Con questo travagliato tristo core Perche so ben che l rio pensier che il core Sempre tien desto condurami a morte', 'Benche fortuna rade volte amica Et la nemica eterna di virtute Invidia d of

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
10,1.063000,0.923071,0.775641
20,1.123000,0.627526,0.775641
30,0.604300,0.552896,0.775641
40,0.625400,0.531255,0.775641
50,0.592300,0.531047,0.775641
60,0.583800,0.513111,0.775641
70,0.483100,0.506955,0.775641
80,0.550900,0.479701,0.775641
90,0.614200,0.502266,0.775641


Counter({0: 121, 1: 35})
              precision    recall  f1-score   support

           0       0.78      1.00      0.87       121
           1       0.00      0.00      0.00        35

    accuracy                           0.78       156
   macro avg       0.39      0.50      0.44       156
weighted avg       0.60      0.78      0.68       156



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classificat